In [2]:
import pandas as pd
from pandas.api.types import is_string_dtype
import numpy as np
import re
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.options.display.max_colwidth = 1000
%matplotlib inline

In [3]:
# Functions
def get_unique_value_col(table, string=False):
    '''
    table: pandas dataframe
    string: computes unique value only for columns containing string
    ''' 
    unique_value = []
    unique_value_str = set()
    for col in table.columns:
        unique_value.append(list(pd.unique(table[col])))
        if string and col != 'patient id':
            unique_value_str.update(set(pd.unique(table[col])))
    
    if string: 
        #even when the column dtype is str, some values are not string
        unique_value_str_clean = unique_value_str.copy()
        for el in unique_value_str:
            if not(isinstance(el, str)):
                unique_value_str_clean.remove(el)
        return(unique_value_str_clean)
    
    return pd.DataFrame({"unique_value": unique_value}, index=table.columns)     


def check_merging(table1, table2, merged, columns_merged):  
    for col in columns_merged:
        NA_table1 = table1[col].isna().sum()
        NA_table2 = table2[col].isna().sum()
        NA_merged = merged[col].isna().sum()
        if not(np.equal((NA_table1 + NA_table2),NA_merged)):
            return print(f"error in {col}")
    if not((table1.shape[0]+table2.shape[0]) == merged.shape[0]):
        return print(f'error in na counts')
    return print('ok')

def get_plot_na(table, axis):
    na_count = table.isna().sum(axis=axis).sort_values(ascending=False)
    if axis==0:
        labels={'index': 'features', 'value':'missing value count'}
        fig = px.bar(na_count, height=600, width=2000, labels=labels)

    else:
        labels={'index': 'number of missing value', 'value':'number of patients'}
        fig = px.bar(na_count.value_counts().sort_index(), height=600, width=2000, labels=labels)

    fig.update_layout(font_size=8, bargap=0.3)
    return fig

YES_REPLACEMENT = 'yes'
NO_REPLACEMENT = 'no'

# Loading

In [4]:
filepath_stroke = "../data_cleaning/clean_data/stroke_clean_v1.csv"
filepath_perfusion = '../data_cleaning/clean_data/perfusion_clean_v1.csv'
filepath_desc = '../data_cleaning/clean_data/description.csv'
df_perfusion = pd.read_csv(filepath_perfusion, header=0)
df_stroke = pd.read_csv(filepath_stroke)
df_desc = pd.read_csv(filepath_desc)

In [5]:
df_stroke.head()

,patient id,completed visit status,htn patient medical history,group,age,height/m,mass/kg,bmi,gender,previous tobacco use,current tobacco use,pack years,years,previous alcohol use,alcohol dose/week,neuropathy autonomic symptoms,dizziness autonomic symptoms,numbness autonomic symptoms,painful feet autonomic symptoms,syncope autonomic symptoms,oh autonomic symptoms,cancer# family history,heartdisease family history,htn family history,dm family history,strokefamily history,cancer patient medical history,atrial fibtrillation patient medical history,heart failure =chf /ifarction=mi patient medical history,hyperlipidemia patient medical history,dm patient medical history,antiplatelets,anticoagulants,antihyperlipidemic,antiparkinsonian,statins,estrogen,ace inhibitors,arbs,beta blockers,diuretics,ca ++ blockers,insulin(yes or no),oral hypoglycemic
0,s0011,ineligible,yes,control,70,1.6383,71.667594,26.701519,f,yes,no,20.0,40.0,yes,0.0,no,no,no,no,no,no,0.0,1.0,0.0,0.0,0.0,no,no,no,no,no,no,no,no,no,no,no,no,yes,no,no,no,no,no
1,s0033,excluded,no,control,65,1.8200,92.986000,28.072093,m,yes,no,10.0,10.0,yes,1.0,no,no,no,no,no,no,0.0,2.0,1.0,1.0,1.0,no,no,no,no,no,no,no,no,no,no,no,yes,no,no,no,no,no,no
2,s0044,completed,no,control,74,1.7018,68.038855,23.493080,f,yes,NaN,40.0,40.0,yes,2.0,no,yes,yes,no,no,yes,3.0,0.0,0.0,0.0,0.0,no,no,no,no,no,no,no,no,no,no,no,no,no,no,no,no,no,no
3,s0067,completed,no,control,70,1.6764,68.492448,24.371788,m,no,no,0.0,0.0,yes,7.0,no,no,no,no,no,no,1.0,0.0,0.0,0.0,1.0,no,no,no,no,no,no,no,no,no,no,no,no,no,no,no,no,no,no
4,s0068,completed,no,control,79,1.5748,64.863709,26.154774,f,no,no,0.0,0.0,no,0.0,no,no,no,no,no,no,3.0,0.0,0.0,0.0,0.0,no,no,no,no,no,no,no,no,no,no,no,no,no,no,no,no,no,no


In [6]:
df_perfusion.head()

,patient id,study,diabetes duration,age,sex,height (m),mass (kg),bmi,diabetes,previous tobacco use,current tobacco use,packyears,years,previous alcohol use,alcohol dose/week,neuropathy autonomic symptoms,dizziness autonomic symptoms,numbness autonomic symptoms,painful feet autonomic symptoms,syncope autonomic symptoms,oh autonomic symptoms,cancer# family history,heartdisease family history,htn family history,dm family history,strokefamily history,cancer patient medical history,stroke patient medical history,atrial fibtrillation patient medical history,heart failure =chf /ifarction=mi patient medical history,hyperlipidemia patient medical history,antiplatelets,anticoagulants,antihyperlipidemic,antiparkinsonian,statins,estrogen,ace inhibitors,arbs,beta blockers,diuretics,ca ++ blockers,insulin(yes or no),oral hypoglycemic
0,s0250,ge75,7.0,50,male,1.790,114.65,35.782279,dm,yes,no,2.86,10.0,yes,7.0,no,no,no,no,no,no,1.0,1.0,1.0,1.0,0.0,no,no,no,no,no,no,no,no,no,no,no,no,no,no,no,no,no,yes
1,s0254,ge75,0.0,69,female,1.531,56.90,24.275148,nondm,yes,no,0.14,1.0,yes,0.5,no,no,no,no,yes,no,1.0,1.0,1.0,0.0,1.0,no,no,no,no,no,no,no,no,no,no,no,no,no,no,no,no,no,no
2,s0255,ge75,3.0,78,female,1.520,65.77,28.466932,dm,no,no,0.00,0.0,yes,9.0,no,no,no,no,no,no,0.0,3.0,0.0,3.0,0.0,no,no,no,no,yes,no,no,no,no,yes,no,no,no,yes,no,no,no,no
3,s0256,ge75,23.0,65,female,1.575,73.50,29.629630,dm,yes,no,5.71,20.0,yes,3.0,no,no,no,no,NaN,no,2.0,1.0,0.0,3.0,1.0,yes,no,no,no,yes,yes,no,no,no,yes,no,no,no,no,no,no,no,yes
4,s0257,ge75,0.0,55,male,1.700,70.31,24.328720,nondm,no,no,0.00,0.0,no,0.0,no,no,no,no,no,no,0.0,1.0,1.0,0.0,1.0,no,no,no,no,no,no,no,no,no,no,no,no,no,no,no,no,no,no


In [7]:
df_desc.head()

,patient id,s0328,s0342,variable description,units
0,NaN,NaN,NaN,NaN,NaN
1,study,ge75,ge75,study number,NaN
2,completed visits status,completed,completed,completed study status,NaN
3,htn patient medical history,no,no,hypertension patient history,NaN
4,group 2,control,dm,group control diabetes,NaN


# Introduction

This jupyter notebook aims to: 
- finalize the data cleaning to find the common features between the stroke and the perfusion datasets
- features relevance for SmartCompanion development
- find protective and risk features through modelling and clustering

The stroke study excludes diabetic patients and the perfusion one excludes people who had a severe stroke. 

To do so we will create a model of stroke prediction based on the stroke dataset and will apply it to the perfusion dataset. Prediction results will be analyze with the clustering results

Problems: 
- check if there's no common patient between the two datasets
- exclude TIA patients
- what to do with illegible patients (having diabetes in the stroke study and a severe stroke in the perfusion one)
- Optimal number of features, features correlations, etc.
- how to deal with mixed data

# Check consistency between the two datasets

In [8]:
df_stroke.shape

(143, 44)

In [9]:
df_perfusion.shape

(87, 44)

In [10]:
common_col = set(df_perfusion.columns) & set(df_stroke.columns)

Let's check if there are common patients between the datasets

In [11]:
set_perfusion = set(df_perfusion["patient id"])
set_stroke = set(df_stroke["patient id"])
# Common patients
print(f'common patient between perfusion and stroke {len(set_perfusion & set_stroke)}')

common patient between perfusion and stroke 0


In [12]:
common_col = set(df_perfusion.columns) & set(df_stroke.columns)
print(len(common_col))
list(common_col)

36


['arbs',
 'syncope autonomic symptoms',
 'statins',
 'patient id',
 'neuropathy autonomic symptoms',
 'beta blockers',
 'current tobacco use',
 'heart failure =chf /ifarction=mi patient medical history',
 'previous tobacco use',
 'ace inhibitors',
 'antiparkinsonian',
 'antiplatelets',
 'oh autonomic symptoms',
 'alcohol dose/week',
 'oral hypoglycemic',
 'age',
 'htn family history',
 'heartdisease family history',
 'antihyperlipidemic',
 'previous alcohol use',
 'strokefamily history',
 'dizziness autonomic symptoms',
 'years',
 'atrial fibtrillation patient medical history',
 'hyperlipidemia patient medical history',
 'cancer# family history',
 'diuretics',
 'numbness autonomic symptoms',
 'ca ++ blockers',
 'painful feet autonomic symptoms',
 'anticoagulants',
 'estrogen',
 'bmi',
 'insulin(yes or no)',
 'cancer patient medical history',
 'dm family history']

36 common features over 44 including demographic, lifestyle and medicall record features

In [13]:
undetected_stroke = df_stroke.columns[~df_stroke.columns.isin(df_perfusion.columns)]
list(undetected_stroke)

['completed visit status',
 'htn patient medical history',
 'group',
 'height/m',
 'mass/kg',
 'gender',
 'pack years',
 'dm patient medical history']

In [14]:
get_unique_value_col(df_stroke[undetected_stroke])

,unique_value
completed visit status,"[ineligible, excluded, completed, v1, lost to followup, excluded pending pcp]"
htn patient medical history,"[yes, no, nan]"
group,"[control, stroke]"
height/m,"[1.6383, 1.82, 1.7018, 1.6764, 1.5748, 1.5494, 1.8288, 1.6256, 1.8034, 1.7526, 1.651, 1.4732, 1.524, 1.49, 1.778, 1.6002, 1.7653, 1.8542, 1.7272, 1.6, 1.57, 1.4986, 1.7, 1.52, 1.502, 1.695, 1.68, 1.715, 1.72, 1.63, 1.8, 1.66, 1.73, 1.75, 1.61, 1.69, 1.64, 1.67, 1.65, 1.59, 1.78, 1.55, 1.47, 1.56, 1.74, nan, 1.83, 1.62, 1.81, 1.53]"
mass/kg,"[71.66759446, 92.986, 68.0388555, 68.49244787, 64.86370891, 52.16312255, 72.5747792, 73.02837157, 95.2543977, 99.7903214, 66.67807839, 80.73944186, 73.93555631, 106.59420695, 54.4310844, 84.82177319, 63.5029318, 53.97749203, 73.48196394, 58.9670081, 67.58526313, 77.1107029, 90.718474, 62.14215469, 92.98643585, 76.20351816, 64.41011654, 60.78137758, 58.05982336, 70.30681735, 67.13167076, 54.88467677, 83.91458845, 57.60623099, 60.32778521, 65.31730128, 113.3980925, 79.37866475, 56.24545388, 114.30527724, 55.33826914, 77.11, 72.57, 45.81, 57.45, 72.25, 69.85, 64.41, 72.95, 91.63, 99.6, 82.9, 83.65, 92.6, 71.55, 112.35, 65.9, 84.5, 81.65, 78.85, 73.7, 66.5, 63.0, 63.25, 74.55, 78.45, 68.3, 75.85, 78.15, 66.2, 69.0, 68.0, 90.7, 51.5, 93.6, 85.9, 49.89, 79.25, 66.3, 83.91, 74.8, 90.72, 93.0, 81.45, 76.4, 94.55, 106.55, nan, 62.15, 96.2, 98.8, 61.0, 77.27, 78.2, 75.38, 85.15, 63.2, 67.9, 107.2, 68.75, ...]"
gender,"[f, m]"
pack years,"[20.0, 10.0, 40.0, 0.0, nan, 72.0, 12.0, 35.0, 15.0, 41.0, 60.0, 5.0, 25.0, 27.0, 160.0, 9.0, 30.0, 11.0, 45.0, 3.75, 48.0, 1.0, 64.0, 43.0, 2.14, 17.0, 30.86, 86.0, 1.71, 62.5, 66.0, 36.0, 0.1667, 10.5, 50.0, 8.0, 33.0, 100.0, 42.0, 1.43, 57.0, 54.0, 96.0, 14.0, 70.0, 6.75, 24.0, 0.6, 4.0]"
dm patient medical history,"[no, nan, yes]"


In [15]:
undetected_perfusion = df_perfusion.columns[~df_perfusion.columns.isin(df_stroke.columns)]
list(undetected_perfusion)

['study',
 'diabetes duration',
 'sex',
 'height (m)',
 'mass (kg)',
 'diabetes',
 'packyears',
 'stroke patient medical history']

In [16]:
get_unique_value_col(df_stroke[undetected_stroke])

,unique_value
completed visit status,"[ineligible, excluded, completed, v1, lost to followup, excluded pending pcp]"
htn patient medical history,"[yes, no, nan]"
group,"[control, stroke]"
height/m,"[1.6383, 1.82, 1.7018, 1.6764, 1.5748, 1.5494, 1.8288, 1.6256, 1.8034, 1.7526, 1.651, 1.4732, 1.524, 1.49, 1.778, 1.6002, 1.7653, 1.8542, 1.7272, 1.6, 1.57, 1.4986, 1.7, 1.52, 1.502, 1.695, 1.68, 1.715, 1.72, 1.63, 1.8, 1.66, 1.73, 1.75, 1.61, 1.69, 1.64, 1.67, 1.65, 1.59, 1.78, 1.55, 1.47, 1.56, 1.74, nan, 1.83, 1.62, 1.81, 1.53]"
mass/kg,"[71.66759446, 92.986, 68.0388555, 68.49244787, 64.86370891, 52.16312255, 72.5747792, 73.02837157, 95.2543977, 99.7903214, 66.67807839, 80.73944186, 73.93555631, 106.59420695, 54.4310844, 84.82177319, 63.5029318, 53.97749203, 73.48196394, 58.9670081, 67.58526313, 77.1107029, 90.718474, 62.14215469, 92.98643585, 76.20351816, 64.41011654, 60.78137758, 58.05982336, 70.30681735, 67.13167076, 54.88467677, 83.91458845, 57.60623099, 60.32778521, 65.31730128, 113.3980925, 79.37866475, 56.24545388, 114.30527724, 55.33826914, 77.11, 72.57, 45.81, 57.45, 72.25, 69.85, 64.41, 72.95, 91.63, 99.6, 82.9, 83.65, 92.6, 71.55, 112.35, 65.9, 84.5, 81.65, 78.85, 73.7, 66.5, 63.0, 63.25, 74.55, 78.45, 68.3, 75.85, 78.15, 66.2, 69.0, 68.0, 90.7, 51.5, 93.6, 85.9, 49.89, 79.25, 66.3, 83.91, 74.8, 90.72, 93.0, 81.45, 76.4, 94.55, 106.55, nan, 62.15, 96.2, 98.8, 61.0, 77.27, 78.2, 75.38, 85.15, 63.2, 67.9, 107.2, 68.75, ...]"
gender,"[f, m]"
pack years,"[20.0, 10.0, 40.0, 0.0, nan, 72.0, 12.0, 35.0, 15.0, 41.0, 60.0, 5.0, 25.0, 27.0, 160.0, 9.0, 30.0, 11.0, 45.0, 3.75, 48.0, 1.0, 64.0, 43.0, 2.14, 17.0, 30.86, 86.0, 1.71, 62.5, 66.0, 36.0, 0.1667, 10.5, 50.0, 8.0, 33.0, 100.0, 42.0, 1.43, 57.0, 54.0, 96.0, 14.0, 70.0, 6.75, 24.0, 0.6, 4.0]"
dm patient medical history,"[no, nan, yes]"


In [17]:
get_unique_value_col(df_perfusion[undetected_perfusion])

,unique_value
study,[ge75]
diabetes duration,"[7.0, 0.0, 3.0, 23.0, 1.0, 26.0, 10.0, 16.0, nan, 4.0, 18.0, 2.0, 15.0, 5.0, 9.0, 21.0, 6.0, 12.0, 17.0, 37.0, 28.0, 25.0, 8.0, 32.0, 31.0, 20.0]"
sex,"[male, female]"
height (m),"[1.79, 1.531, 1.52, 1.575, 1.7, 1.5, 1.722, 1.68, 1.73, 1.807, 1.63, 1.6, 1.608, 1.62, 1.602, 1.54, 1.57, 1.66, 1.85, 1.46, 1.9, 1.59, 1.84, 1.64, 1.8, 1.75, 1.53, 1.76, 1.83, 1.72, 1.56, 1.65, 1.78, 1.71, 1.82, 1.74, 1.67, 1.77, 1.55, 1.61, 1.92, 1.69, 1.561, 1.557, 1.818, 1.768, 1.667, 1.727, 1.798, nan]"
mass (kg),"[114.65, 56.9, 65.77, 73.5, 70.31, 83.46, 98.4, 85.28, 96.16, 71.67, 85.8, 63.5, 81.65, 56.25, 69.2, 48.99, 52.25, 65.65, 54.43, 88.8, 64.75, 74.6, 84.1, 95.4, 91.4, 65.2, 94.65, 74.25, 79.6, 84.0, 69.5, 124.0, 79.8, 96.95, 91.3, 75.85, 85.0, 81.95, 77.95, 99.05, 84.9, 83.5, 64.35, 95.25, 83.1, 84.05, 130.7, 86.05, 89.6, 122.5, 65.35, 63.8, 91.1, 81.15, 119.4, 77.5, 86.1, 60.5, 100.3, 78.8, 93.55, 121.6, 68.0, 69.0, 72.7, 69.85, 85.75, 63.96, 86.85, 85.3, 87.8, 89.65, 86.95, 75.15, 64.65, 79.5, 133.05, 92.85, 72.3, 80.74, 75.4, 93.6, 71.05, nan]"
diabetes,"[dm, nondm]"
packyears,"[2.86, 0.14, 0.0, 5.71, 34.0, 20.0, 10.0, 15.0, 60.0, 30.0, 25.0, 4.29, nan, 35.0, 12.5, 8.57, 57.0, 80.0, 26.0, 40.0, 61.5, 48.0, 45.0, 7.5, 1.14, 1.25, 0.5, 0.29]"
stroke patient medical history,"[no, tia]"


The only uncommon features are
- perfusion dataset: 'study'
- stroke dataset: "completed visit status", "htn patient medical history"
Regarding diabetes duration we are keeping this feature at the moment. We will try different combination of features for the clustering and the model prediction

We can 
- drop those features 
- change the spelling of the others so they match
- change the name of the categories 


In [18]:
df_perfusion_v1 = df_perfusion.drop(labels='study', axis="columns")
df_stroke_v1 = df_stroke.drop(labels=['completed visit status', 'htn patient medical history'], axis="columns")

In [19]:
dict_rename_perfusion = {
    "sex" : "gender", 
    "height (m)": "height",
    "mass (kg)": "mass",
    "packyears": "pack years", 
}

df_perfusion_v1.rename(mapper=dict_rename_perfusion, axis=1, inplace=True)

In [20]:
dict_rename_stroke = {
    "group" : "stroke patient medical history", 
    "height/m": "height",
    "mass/kg": "mass",
    "dm patient medical history": "diabetes", 
}
df_stroke_v1.rename(mapper=dict_rename_stroke, axis=1, inplace=True)

In [21]:
set(df_perfusion_v1.columns) & set(df_stroke_v1.columns)

{'ace inhibitors',
 'age',
 'alcohol dose/week',
 'anticoagulants',
 'antihyperlipidemic',
 'antiparkinsonian',
 'antiplatelets',
 'arbs',
 'atrial fibtrillation patient medical history',
 'beta blockers',
 'bmi',
 'ca ++ blockers',
 'cancer patient medical history',
 'cancer# family history',
 'current tobacco use',
 'diabetes',
 'diuretics',
 'dizziness autonomic symptoms',
 'dm family history',
 'estrogen',
 'gender',
 'heart failure =chf /ifarction=mi patient medical history',
 'heartdisease family history',
 'height',
 'htn family history',
 'hyperlipidemia patient medical history',
 'insulin(yes or no)',
 'mass',
 'neuropathy autonomic symptoms',
 'numbness autonomic symptoms',
 'oh autonomic symptoms',
 'oral hypoglycemic',
 'pack years',
 'painful feet autonomic symptoms',
 'patient id',
 'previous alcohol use',
 'previous tobacco use',
 'statins',
 'stroke patient medical history',
 'strokefamily history',
 'syncope autonomic symptoms',
 'years'}

Let's check consistency of values between the two datasets

In [22]:
unique_value_stroke = []
unique_value_perfusion = []
symmetric_difference = []

for col in (set(df_perfusion_v1.columns) & set(df_stroke_v1.columns)):
    unique_stroke = list(pd.unique(df_stroke_v1[col]))
    unique_perfusion = list(pd.unique(df_perfusion_v1[col]))
    unique_value_stroke.append(unique_stroke)
    unique_value_perfusion.append(unique_perfusion)
    symmetric_difference.append((set(unique_stroke).symmetric_difference(set(unique_perfusion))))
          
df_common_features_comparison = pd.DataFrame({'stroke':unique_value_stroke, 'perfusion':unique_value_perfusion, "symmetric_diff": symmetric_difference}, index=(set(df_perfusion_v1.columns) & set(df_stroke_v1.columns)))
df_common_features_comparison

,stroke,perfusion,symmetric_diff
arbs,"[yes, no, nan]","[no, yes, nan]",{}
syncope autonomic symptoms,"[no, yes, nan]","[no, yes, nan]",{}
statins,"[no, yes, nan]","[no, yes, nan]",{}
patient id,"[s0011, s0033, s0044, s0067, s0068, s0078, s0098, s0121, s0132, s0147, s0151, s0153, s0154, s0157, s0160, s0162, s0163, s0164, s0165, s0166, s0169, s0172, s0174, s0175, s0176, s0177, s0178, s0181, s0182, s0183, s0184, s0185, s0187, s0188, s0194, s0197, s0200, s0203, s0204, s0205, s0206, s0207, s0208, s0209, s0210, s0211, s0212, s0213, s0215, s0216, s0217, s0218, s0219, s0220, s0221, s0222, s0223, s0225, s0226, s0227, s0228, s0229, s0231, s0232, s0233, s0235, s0236, s0238, s0239, s0240, s0241, s0242, s0243, s0244, s0245, s0246, s0247, s0248, s0249, s0261, s0275, s0277, s0295, s0305, s0307, s0313, s0319, s0321, s0322, s0324, s0325, s0329, s0331, s0332, s0334, s0335, s0336, s0337, s0338, s0340, ...]","[s0250, s0254, s0255, s0256, s0257, s0262, s0264, s0267, s0270, s0271, s0273, s0274, s0279, s0281, s0282, s0283, s0287, s0288, s0290, s0292, s0294, s0296, s0300, s0301, s0302, s0303, s0304, s0306, s0308, s0310, s0312, s0314, s0315, s0316, s0317, s0318, s0320, s0323, s0326, s0327, s0328, s0330, s0333, s0339, s0342, s0349, s0360, s0365, s0366, s0368, s0372, s0375, s0381, s0382, s0386, s0390, s0391, s0392, s0393, s0395, s0398, s0403, s0405, s0406, s0407, s0408, s0409, s0410, s0411, s0416, s0417, s0420, s0421, s0422, s0423, s0424, s0426, s0427, s0430, s0431, s0432, s0433, s0434, s0435, s0438, s0441, s0443]","{s0341, s0371, s0406, s0390, s0169, s0336, s0302, s0255, s0330, s0187, s0416, s0402, s0166, s0338, s0235, s0366, s0380, s0404, s0287, s0411, s0165, s0177, s0290, s0068, s0262, s0203, s0164, s0353, s0432, s0267, s0211, s0333, s0347, s0328, s0162, s0369, s0151, s0326, s0422, s0271, s0433, s0200, s0315, s0175, s0098, s0282, s0409, s0408, s0229, s0249, s0182, s0256, s0414, s0427, s0240, s0275, s0217, s0434, s0227, s0241, s0153, s0359, s0247, s0435, s0374, s0342, s0423, s0443, s0270, s0301, s0379, s0382, s0383, s0243, s0375, s0210, s0391, s0365, s0209, s0223, s0410, s0332, s0185, s0277, s0274, s0314, s0356, s0329, s0239, s0334, s0354, s0163, s0424, s0281, s0364, s0296, s0393, s0245, s0324, s0322, ...}"
neuropathy autonomic symptoms,"[no, yes, nan]","[no, yes]",{nan}
beta blockers,"[no, yes, nan]","[no, yes, nan]",{}
current tobacco use,"[no, nan, yes]","[no, yes, nan]",{}
heart failure =chf /ifarction=mi patient medical history,"[no, nan, yes]","[no, nan, yes]",{}
previous tobacco use,"[yes, no, nan]","[yes, no, nan]",{}
ace inhibitors,"[no, yes, nan]","[no, yes, nan]",{}


**Inconsistency**:
stroke patient medical history: control/stroke
numbness autonomic symptoms: 20 -> let'sdrop this patient
gender: f, m, male, female
diabetes: yes, no and dm non dm

In [23]:
df_stroke_v1[df_stroke_v1["numbness autonomic symptoms"] == "20"]

,patient id,stroke patient medical history,age,height,mass,bmi,gender,previous tobacco use,current tobacco use,pack years,years,previous alcohol use,alcohol dose/week,neuropathy autonomic symptoms,dizziness autonomic symptoms,numbness autonomic symptoms,painful feet autonomic symptoms,syncope autonomic symptoms,oh autonomic symptoms,cancer# family history,heartdisease family history,htn family history,dm family history,strokefamily history,cancer patient medical history,atrial fibtrillation patient medical history,heart failure =chf /ifarction=mi patient medical history,hyperlipidemia patient medical history,diabetes,antiplatelets,anticoagulants,antihyperlipidemic,antiparkinsonian,statins,estrogen,ace inhibitors,arbs,beta blockers,diuretics,ca ++ blockers,insulin(yes or no),oral hypoglycemic
129,s0380,stroke,69,1.74,85.15,28.124587,m,yes,no,70.0,35.0,yes,20.0,yes,no,20,yes,no,no,0.0,1.0,3.0,0.0,2.0,no,no,no,yes,no,yes,no,no,no,yes,no,yes,no,no,no,yes,no,no


In [24]:
df_stroke_v1.drop(labels=df_stroke_v1[df_stroke_v1["numbness autonomic symptoms"] == "20"].index, inplace=True)

In [25]:
no_re = re.compile("(control)|(^nondm$)")
yes_re = re.compile("(^dm$)|(stroke)")
gender_female_re = re.compile("(^f$)|(female)")
gender_male_re = re.compile("(^m$)|(male)")

# Let's start with the stroke dataset
print(f"'yes' regex results: {list(filter(yes_re.match, get_unique_value_col(df_stroke_v1, string=True)))}")
print(f"'no' regex results: {list(filter(no_re.match, get_unique_value_col(df_stroke_v1, string=True)))}")
print(f"'female' regex results: {list(filter(gender_female_re.match, get_unique_value_col(df_stroke_v1, string=True)))}")
print(f"'male' regex results: {list(filter(gender_male_re.match, get_unique_value_col(df_stroke_v1, string=True)))}")

'yes' regex results: ['stroke']
'no' regex results: ['control']
'female' regex results: ['f']
'male' regex results: ['m']


In [26]:
df_stroke_v1 = df_stroke_v1.replace(to_replace = {yes_re:YES_REPLACEMENT, 
                              no_re: NO_REPLACEMENT, 
                              gender_female_re: 'female',
                              gender_male_re: 'male'}, 
                              regex=True)

In [27]:
print(f"'yes' regex results: {list(filter(yes_re.match, get_unique_value_col(df_perfusion_v1, string=True)))}")
print(f"'no' regex results: {list(filter(no_re.match, get_unique_value_col(df_perfusion_v1, string=True)))}")
print(f"'female' regex results: {list(filter(gender_female_re.match, get_unique_value_col(df_perfusion_v1, string=True)))}")
print(f"'male' regex results: {list(filter(gender_male_re.match, get_unique_value_col(df_perfusion_v1, string=True)))}")

'yes' regex results: ['dm']
'no' regex results: ['nondm']
'female' regex results: ['female']
'male' regex results: ['male']


In [28]:
df_perfusion_v1 = df_perfusion_v1.replace(to_replace = {yes_re:YES_REPLACEMENT, 
                              no_re: NO_REPLACEMENT, 
                              gender_female_re: 'female',
                              gender_male_re: 'male'}, 
                              regex=True)

In [29]:
print(f"'yes' regex results: {list(filter(yes_re.match, get_unique_value_col(df_perfusion_v1, string=True)))}")
print(f"'no' regex results: {list(filter(no_re.match, get_unique_value_col(df_perfusion_v1, string=True)))}")
print(f"'female' regex results: {list(filter(gender_female_re.match, get_unique_value_col(df_perfusion_v1, string=True)))}")
print(f"'male' regex results: {list(filter(gender_male_re.match, get_unique_value_col(df_perfusion_v1, string=True)))}")

'yes' regex results: []
'no' regex results: []
'female' regex results: ['female']
'male' regex results: ['male']


In [30]:
unique_value_stroke = []
unique_value_perfusion = []
symmetric_difference = []

for col in (set(df_perfusion_v1.columns) & set(df_stroke_v1.columns)):
    unique_stroke = list(pd.unique(df_stroke_v1[col]))
    unique_perfusion = list(pd.unique(df_perfusion_v1[col]))
    unique_value_stroke.append(unique_stroke)
    unique_value_perfusion.append(unique_perfusion)
    symmetric_difference.append((set(unique_stroke).symmetric_difference(set(unique_perfusion))))
          
df_common_features_comparison = pd.DataFrame({'stroke':unique_value_stroke, 'perfusion':unique_value_perfusion, "symmetric_diff": symmetric_difference}, index=(set(df_perfusion_v1.columns) & set(df_stroke_v1.columns)))
df_common_features_comparison

,stroke,perfusion,symmetric_diff
arbs,"[yes, no, nan]","[no, yes, nan]",{}
syncope autonomic symptoms,"[no, yes, nan]","[no, yes, nan]",{}
statins,"[no, yes, nan]","[no, yes, nan]",{}
patient id,"[s0011, s0033, s0044, s0067, s0068, s0078, s0098, s0121, s0132, s0147, s0151, s0153, s0154, s0157, s0160, s0162, s0163, s0164, s0165, s0166, s0169, s0172, s0174, s0175, s0176, s0177, s0178, s0181, s0182, s0183, s0184, s0185, s0187, s0188, s0194, s0197, s0200, s0203, s0204, s0205, s0206, s0207, s0208, s0209, s0210, s0211, s0212, s0213, s0215, s0216, s0217, s0218, s0219, s0220, s0221, s0222, s0223, s0225, s0226, s0227, s0228, s0229, s0231, s0232, s0233, s0235, s0236, s0238, s0239, s0240, s0241, s0242, s0243, s0244, s0245, s0246, s0247, s0248, s0249, s0261, s0275, s0277, s0295, s0305, s0307, s0313, s0319, s0321, s0322, s0324, s0325, s0329, s0331, s0332, s0334, s0335, s0336, s0337, s0338, s0340, ...]","[s0250, s0254, s0255, s0256, s0257, s0262, s0264, s0267, s0270, s0271, s0273, s0274, s0279, s0281, s0282, s0283, s0287, s0288, s0290, s0292, s0294, s0296, s0300, s0301, s0302, s0303, s0304, s0306, s0308, s0310, s0312, s0314, s0315, s0316, s0317, s0318, s0320, s0323, s0326, s0327, s0328, s0330, s0333, s0339, s0342, s0349, s0360, s0365, s0366, s0368, s0372, s0375, s0381, s0382, s0386, s0390, s0391, s0392, s0393, s0395, s0398, s0403, s0405, s0406, s0407, s0408, s0409, s0410, s0411, s0416, s0417, s0420, s0421, s0422, s0423, s0424, s0426, s0427, s0430, s0431, s0432, s0433, s0434, s0435, s0438, s0441, s0443]","{s0341, s0371, s0406, s0390, s0169, s0336, s0302, s0255, s0330, s0187, s0416, s0402, s0166, s0338, s0235, s0366, s0404, s0287, s0411, s0165, s0177, s0290, s0068, s0262, s0203, s0164, s0353, s0432, s0267, s0211, s0333, s0347, s0328, s0162, s0369, s0151, s0326, s0422, s0271, s0433, s0200, s0315, s0175, s0098, s0282, s0409, s0408, s0229, s0249, s0182, s0256, s0414, s0427, s0240, s0275, s0217, s0434, s0227, s0241, s0153, s0359, s0247, s0435, s0374, s0342, s0423, s0443, s0270, s0301, s0379, s0382, s0383, s0243, s0375, s0210, s0391, s0365, s0209, s0223, s0410, s0332, s0185, s0277, s0274, s0314, s0356, s0329, s0239, s0334, s0354, s0163, s0424, s0281, s0364, s0296, s0393, s0245, s0324, s0322, s0250, ...}"
neuropathy autonomic symptoms,"[no, yes, nan]","[no, yes]",{nan}
beta blockers,"[no, yes, nan]","[no, yes, nan]",{}
current tobacco use,"[no, nan, yes]","[no, yes, nan]",{}
heart failure =chf /ifarction=mi patient medical history,"[no, nan, yes]","[no, nan, yes]",{}
previous tobacco use,"[yes, no, nan]","[yes, no, nan]",{}
ace inhibitors,"[no, yes, nan]","[no, yes, nan]",{}


# Feature Selection

From the pandas profiling (exploration notebooks) we know that "antiparkinsonian" and "insuline(yes or no)" only have 'no's for the stroke dataset so there are not usable in our case. 
Because we may extract some information from the clustering, we are keeping a trace of features droped in stroke but kept in perfusion 

In [31]:
clustering_features = ["insulin(yes or no)", "antiparkinsonian"]  

In [32]:
df_stroke_v1 = df_stroke_v1.drop(labels=clustering_features, axis="columns")

In [33]:
df_stroke_v1.shape

(142, 40)

There are 40 features in the stroke datasets, the following questions have to be adressed: 
- Are 40 features too many features for 142 patients?
- how features are correlated?
- missing value?
- data type

All of this questions depend on the chosen algorithm. 

Random Forest may seem a good start:
 - support mixed type 
 - doesn't need normalization for numerical features
 - can be used as a feature selection algorithm
 - handle quite well correlated/redundant variables,
 
The only problem is missing value so we'll have to drop some features and patients

In [34]:
df_stroke.shape

(143, 44)

In [35]:
get_plot_na(df_stroke_v1, 1)

In [36]:
na_patients = df_stroke_v1.isna().sum(1)[df_stroke_v1.isna().sum(1) == 33].index

In [37]:
df_stroke_v1.drop(na_patients,inplace=True)

Among the top features with missing values 4 describe tobacco use
- pack years
- years
- current tobacco use
- previous tobacco use
What matters for cardiovascular risks is not the quantity but for how long you've been smoking.
- let's keep years and discard other features
- before that let's put 0 for patient how has never smoke

In [38]:
df_stroke_v1[df_stroke_v1["years"].isna()][["years", "previous tobacco use", "current tobacco use"]]

,years,previous tobacco use,current tobacco use
9,NaN,yes,NaN
20,NaN,yes,no
27,NaN,yes,yes
39,NaN,yes,no
62,NaN,no,no
63,NaN,no,no
66,NaN,no,no
84,NaN,yes,yes
106,NaN,yes,yes
111,NaN,yes,yes


In [39]:
df_stroke_v1.drop((df_stroke_v1[(df_stroke_v1["previous tobacco use"] == 'no')
                       & (df_stroke_v1["current tobacco use"] == 'no')
                       & (df_stroke_v1["years"] == 40)]).index , 
        inplace=True)

Let's drop the patient with 40 years of smoking but no current of previous tobacco use

In [40]:
years_na = df_stroke_v1[(df_stroke_v1["previous tobacco use"] == 'no')
                       & (df_stroke_v1["current tobacco use"] == 'no')
                       & (df_stroke_v1["years"] != 0)].index
df_stroke_v1.loc[years_na, "years"] = 0

In [41]:
df_stroke_v1.drop(labels=["pack years", "current tobacco use", "previous tobacco use"], axis="columns", inplace=True)

In [42]:
df_stroke_v1[df_stroke_v1["years"].isna()].index

Int64Index([9, 20, 27, 39, 84, 106, 111, 139], dtype='int64')

In [43]:
df_stroke_v1.drop(labels = df_stroke_v1[df_stroke_v1["years"].isna()].index, 
                 inplace=True)

In [44]:
df_stroke_v1.shape

(132, 37)

In [45]:
get_plot_na(df_stroke_v1, 0)

As for alcohol there are multiple feaures describing alcohol consumption 
- alcohol dose/week
- previous alchohol use

Now there is around 20 patients with missing value, let's drop them at the moment and test random forest

In [46]:
df_stroke_v1.dropna(axis='index', inplace=True)

In [47]:
get_plot_na(df_stroke_v1,0)

In [48]:
get_plot_na(df_perfusion_v1,0)

Let's do the same for the perfusion dataset: drop patient with missing value

In [49]:
df_perfusion_v1.dropna(inplace=True)
df_stroke_v1.set_index("patient id", drop=True, inplace=True)
df_perfusion_v1.set_index("patient id", drop=True, inplace=True)

In [50]:
df_perfusion_v1.shape

(56, 42)

In [51]:
df_perfusion_v1["diabetes"].value_counts()

yes    50
no      6
Name: diabetes, dtype: int64

In [52]:
#df_perfusion_v1.to_csv("../data_cleaning/clean_data/df_perfusion_v2.csv", sep=",", header=True, index=True)
#df_stroke_v1.to_csv("../data_cleaning/clean_data/df_stroke_v2.csv", sep=",", header=True, index=True)